In [13]:
import cv2
import numpy as np
import os

class PersonCounterDL:
    """
    Counts people using OpenCV's deep learning face detector
    (DNN module with ResNet-based model - much more robust than Haar Cascade)
    """
    
    def __init__(self):
        """
        Initialize face and eye detectors using pre-trained Haar Cascades
        """
        cascade_path = cv2.data.haarcascades
        
        self.face_cascade = cv2.CascadeClassifier(
            os.path.join(cascade_path, 'haarcascade_frontalface_default.xml')
        )
        self.eye_cascade = cv2.CascadeClassifier(
            os.path.join(cascade_path, 'haarcascade_eye.xml')
        )
        
        if self.face_cascade.empty():
            raise RuntimeError("Could not load Haar Cascade face detector")
        if self.eye_cascade.empty():
            print("⚠ Warning: Eye cascade not available (non-critical)")
        
        self.use_eye_detection = not self.eye_cascade.empty()
        print("✓ Haar Cascade face detector loaded")
    

    def _detect_faces_haar(self, image):
        """
        Fallback: Haar Cascade detection with validation
        """
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        gray = cv2.equalizeHist(gray)
        
        faces = self.face_cascade.detectMultiScale(
            gray,
            scaleFactor=1.05,
            minNeighbors=3,
            minSize=(20, 20),
            maxSize=(300, 300)
        )
        
        # Validate detections to reduce false positives
        validated_faces = []
        for (x, y, w, h) in faces:
            if self._is_valid_face(image, gray, x, y, w, h):
                validated_faces.append((x, y, w, h, 0.9))
        
        return validated_faces
    
    def _is_valid_face(self, image, gray, x, y, w, h):
        """
        Validate if detection is likely a real face
        Checks: aspect ratio, skin color, edges, and optional eye detection
        """
        # 1. Check aspect ratio - face should be roughly square
        aspect_ratio = float(w) / h if h > 0 else 0
        if not (0.6 < aspect_ratio < 1.4):  # Face is roughly square
            return False
        
        # 2. Check for skin tone in the region (most reliable)
        roi_bgr = image[y:y+h, x:x+w]
        roi_hsv = cv2.cvtColor(roi_bgr, cv2.COLOR_BGR2HSV)
        
        # Count pixels that look like skin (HSV range)
        lower_skin = np.array([0, 20, 70], dtype=np.uint8)
        upper_skin = np.array([20, 255, 255], dtype=np.uint8)
        skin_mask1 = cv2.inRange(roi_hsv, lower_skin, upper_skin)
        
        lower_skin2 = np.array([170, 20, 70], dtype=np.uint8)
        upper_skin2 = np.array([180, 255, 255], dtype=np.uint8)
        skin_mask2 = cv2.inRange(roi_hsv, lower_skin2, upper_skin2)
        
        skin_mask = cv2.bitwise_or(skin_mask1, skin_mask2)
        skin_percentage = np.count_nonzero(skin_mask) / (w * h)
        
        if skin_percentage < 0.15:  # Should have some skin color
            return False
        
        # 3. Check for edges (faces have texture, not smooth surfaces)
        roi_gray = gray[y:y+h, x:x+w]
        edges = cv2.Canny(roi_gray, 50, 150)
        edge_percentage = np.count_nonzero(edges) / (w * h)
        
        if edge_percentage < 0.05:  # Should have some edges/texture
            return False
        
        # 4. Check contrast - faces have good contrast
        contrast = roi_gray.std()
        if contrast < 15:  # Low contrast = likely not a real face
            return False
        
        # 5. Optional: Eye detection as bonus check (not required)
        # Eyes are helpful but not every face angle detects eyes well
        if self.use_eye_detection:
            eyes = self.eye_cascade.detectMultiScale(roi_gray)
            # If we detect eyes, great! But don't fail if we don't
            # (some angles/lighting won't detect eyes)
        
        return True
    
    def _detect_edge_based_people(self, image):
        """
        Fallback: edge-based detection for full body shots
        Useful when face not visible or too small
        """
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        # Edge detection
        edges = cv2.Canny(gray, 50, 150)
        
        # Morphological operations
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (20, 50))
        morph = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel, iterations=2)
        
        # Find contours
        contours, _ = cv2.findContours(morph, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        people = []
        h, w = image.shape[:2]
        
        for contour in contours:
            area = cv2.contourArea(contour)
            # Person-sized objects
            if 2000 < area < (h * w * 0.4):
                x, y, bw, bh = cv2.boundingRect(contour)
                aspect = float(bw) / bh if bh > 0 else 0
                
                # Person-like aspect ratio (taller than wide)
                if 0.2 < aspect < 0.8:
                    people.append((x, y, bw, bh, 0.5))
        
        return people
    
    def _nms(self, boxes, overlap_thresh=0.3):
        """
        Non-maximum suppression: merge overlapping boxes
        """
        if not boxes:
            return []
        
        boxes = sorted(boxes, key=lambda b: b[4], reverse=True)  # Sort by confidence
        keep = []
        
        while boxes:
            current = boxes.pop(0)
            keep.append(current)
            
            remaining = []
            for box in boxes:
                x1, y1, w1, h1, conf1 = current
                x2, y2, w2, h2, conf2 = box
                
                # Calculate IoU (Intersection over Union)
                xi1, yi1 = max(x1, x2), max(y1, y2)
                xi2, yi2 = min(x1 + w1, x2 + w2), min(y1 + h1, y2 + h2)
                
                if xi2 > xi1 and yi2 > yi1:
                    inter = (xi2 - xi1) * (yi2 - yi1)
                    union = w1 * h1 + w2 * h2 - inter
                    iou = inter / union if union > 0 else 0
                    
                    if iou <= overlap_thresh:
                        remaining.append(box)
                else:
                    remaining.append(box)
            
            boxes = remaining
        
        return keep
    
    def count_persons(self, image_path, visualize=True, confidence_thresh=0.5):
        """
        Main function: detect and count people
        Uses multiple detection methods for robustness
        
        Args:
            image_path: Path to image
            visualize: Return annotated image
            confidence_thresh: Confidence threshold for deep learning detector
        
        Returns:
            count: Number of detected people
            annotated: Annotated image
            detections: List of detection boxes
        """
        if not os.path.exists(image_path):
            raise FileNotFoundError(f"Image not found: {image_path}")
        
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError(f"Cannot read image: {image_path}")
        
        # Resize if too large
        h, w = image.shape[:2]
        if w > 1280:
            scale = 1280 / w
            image = cv2.resize(image, (int(w * scale), int(h * scale)))
        
        all_detections = []
        detection_methods = []
        
        # Method 1: Haar Cascade with face validation
        haar_faces = self._detect_faces_haar(image)
        all_detections.extend(haar_faces)
        detection_methods.append(f"Haar+Validation: {len(haar_faces)}")
        
        # Method 3: Edge-based detection (for full body)
        edge_people = self._detect_edge_based_people(image)
        all_detections.extend(edge_people)
        detection_methods.append(f"Edge: {len(edge_people)}")
        
        # Apply NMS to remove duplicates
        final_detections = self._nms(all_detections, overlap_thresh=0.3)
        final_detections = sorted(final_detections, key=lambda x: x[0])  # Sort by x position
        
        count = len(final_detections)
        
        # Visualization
        annotated = image.copy() if visualize else None
        
        if visualize:
            for idx, (x, y, bw, bh, conf) in enumerate(final_detections, 1):
                # Draw bounding box
                cv2.rectangle(annotated, (x, y), (x + bw, y + bh), (0, 255, 0), 2)
                
                # Draw centroid
                cx, cy = x + bw // 2, y + bh // 2
                cv2.circle(annotated, (cx, cy), 5, (0, 0, 255), -1)
                
                # Label with confidence
                label = f"Person {idx} ({conf:.2f})"
                cv2.putText(annotated, label, (x, y - 10),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
            
            # Total count
            cv2.putText(annotated, f"Total: {count} person(s)", (10, 30),
                       cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), 2)
            
            # Detection methods used
            y_offset = 70
            for method in detection_methods:
                cv2.putText(annotated, method, (10, y_offset),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 200, 0), 1)
                y_offset += 25
        
        return count, annotated, final_detections


def main(image_path="test_image.jpg", save_output=True, confidence=0.5):
    """
    Main execution
    """
    counter = PersonCounterDL()
    
    if not os.path.exists(image_path):
        print(f"❌ Image not found: {image_path}")
        return
    
    try:
        count, annotated, detections = counter.count_persons(
            image_path, visualize=True, confidence_thresh=confidence
        )
        
        print(f"\n{'='*60}")
        print(f"PERSON DETECTION RESULTS")
        print(f"{'='*60}")
        print(f"✓ Total persons detected: {count}")
        
        if detections:
            print(f"\nDetailed information:")
            for idx, (x, y, w, h, conf) in enumerate(detections, 1):
                print(f"  Person {idx}:")
                print(f"    - Position: ({x}, {y})")
                print(f"    - Size: {w}x{h} pixels")
                print(f"    - Confidence: {conf:.2f}")
                print(f"    - Center: ({x + w//2}, {y + h//2})")
        
        # Save result
        if save_output:
            # Add -1 suffix instead of overwriting
            base_name = image_path.rsplit('.', 1)[0]
            ext = image_path.rsplit('.', 1)[1] if '.' in image_path else 'jpg'
            output_path = f"{base_name}-1.{ext}"
            
            # Handle multiple saves (if -1 exists, try -2, -3, etc.)
            counter_suffix = 1
            while os.path.exists(output_path):
                counter_suffix += 1
                output_path = f"{base_name}-{counter_suffix}.{ext}"
            
            cv2.imwrite(output_path, annotated)
            print(f"\n✓ Result saved: {output_path}")
        
        return annotated, count, detections
    
    except Exception as e:
        print(f"❌ Error: {e}")
        return None, 0, []


if __name__ == "__main__":
    # Try with different confidence thresholds if needed
    result_img, person_count, details = main(image_path="test_image.jpg", confidence=0.4)

✓ Haar Cascade face detector loaded

PERSON DETECTION RESULTS
✓ Total persons detected: 13

Detailed information:
  Person 1:
    - Position: (82, 288)
    - Size: 67x67 pixels
    - Confidence: 0.90
    - Center: (115, 321)
  Person 2:
    - Position: (88, 341)
    - Size: 67x67 pixels
    - Confidence: 0.90
    - Center: (121, 374)
  Person 3:
    - Position: (206, 318)
    - Size: 60x60 pixels
    - Confidence: 0.90
    - Center: (236, 348)
  Person 4:
    - Position: (269, 868)
    - Size: 61x61 pixels
    - Confidence: 0.90
    - Center: (299, 898)
  Person 5:
    - Position: (362, 331)
    - Size: 62x62 pixels
    - Confidence: 0.90
    - Center: (393, 362)
  Person 6:
    - Position: (412, 233)
    - Size: 37x37 pixels
    - Confidence: 0.90
    - Center: (430, 251)
  Person 7:
    - Position: (532, 337)
    - Size: 70x70 pixels
    - Confidence: 0.90
    - Center: (567, 372)
  Person 8:
    - Position: (661, 301)
    - Size: 64x64 pixels
    - Confidence: 0.90
    - Center: (69